In [4]:
# --- Final Script to analyze optimal location for Energy Island with MCDA ---
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import geodesic

# --- Step 1: Load all datasets ---
solar = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\solar_mcda_scores.csv")
wind = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\wind_mcda_output.txt")
land = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\landcover_mcda_scores.txt")              # TXT
grid = pd.read_csv(r"C:\GISDataManipulation\StationTranmissionLineCombined\merged_grid_infra.txt")               # TXT

# --- Step 2: Round coordinates and convert to numpy arrays ---
solar_coords = solar[['Latitude', 'Longitude']].to_numpy()
wind_coords = wind[['Latitude', 'Longitude']].to_numpy()
land_coords = land[['Latitude', 'Longitude']].to_numpy()

# --- Step 3: Build KDTree from solar coordinates (as base) ---
tree_wind = cKDTree(wind_coords)
tree_land = cKDTree(land_coords)

# Match wind to solar (get nearest index)
_, idx_wind = tree_wind.query(solar_coords, k=1, distance_upper_bound=0.01)  # ~1km tolerance
_, idx_land = tree_land.query(solar_coords, k=1, distance_upper_bound=0.01)

# Filter valid indices (drop unmatched)
valid_mask = (idx_wind < len(wind)) & (idx_land < len(land))
solar = solar[valid_mask].reset_index(drop=True)
wind_matched = wind.iloc[idx_wind[valid_mask]].reset_index(drop=True)
land_matched = land.iloc[idx_land[valid_mask]].reset_index(drop=True)

# --- Step 4: Combine datasets ---
merged = pd.DataFrame({
    'Latitude': solar['Latitude'],
    'Longitude': solar['Longitude'],
    'solar_score': solar['solar_score'],
    'wind_score': wind_matched['wind_score'],
    'land_score': land_matched['land_score']
})

print("Matched rows:", len(merged))

# --- Step 5: Remove invalid coordinates ---
merged = merged.dropna(subset=['Latitude', 'Longitude'])
merged = merged[pd.to_numeric(merged['Latitude'], errors='coerce').notnull()]
merged = merged[pd.to_numeric(merged['Longitude'], errors='coerce').notnull()]
merged['Latitude'] = merged['Latitude'].astype(float)
merged['Longitude'] = merged['Longitude'].astype(float)

# --- Step 6: Compute distance to grid ---
print("Calculating distance to nearest grid point...")
# Convert coordinates to radians for basic approximation x
def latlon_to_xy(lat, lon):
    # approximate conversion: 1 deg ≈ 111 km
    return np.radians(lat) * 6371, np.radians(lon) * 6371 * np.cos(np.radians(lat))

# Convert merged sample points to approximate XY
sample_xy = np.array([latlon_to_xy(lat, lon) for lat, lon in zip(merged['Latitude'], merged['Longitude'])])

# Convert grid points to approximate XY
grid_xy = np.array([latlon_to_xy(lat, lon) for lat, lon in zip(grid['Latitude'], grid['Longitude'])])

# Build KDTree for fast nearest-neighbor search
tree = cKDTree(grid_xy)

# Query nearest distance
dists, _ = tree.query(sample_xy, k=1)
merged['distance_to_grid_km'] = dists
merged = merged.dropna(subset=['distance_to_grid_km'])

# --- Step 7: Normalize and score grid distance ---
d_min = merged['distance_to_grid_km'].min()
d_max = merged['distance_to_grid_km'].max()
merged['grid_norm'] = (d_max - merged['distance_to_grid_km']) / (d_max - d_min)
merged['grid_score'] = (merged['grid_norm'] * 9).round().astype(int)

# --- Step 8: Weighted MCDA score ---
merged['final_score'] = (
    0.35 * merged['solar_score'] +
    0.25 * merged['wind_score'] +
    0.20 * merged['land_score'] +
    0.20 * merged['grid_score']
).round(2)

# --- Step 9: Save result ---
output = merged[['Latitude', 'Longitude', 'solar_score', 'wind_score', 'land_score', 'grid_score', 'final_score']]
output.to_csv(r"C:\GISDataManipulation\MCDA Files\FinalMCDAScore\final_mcda_scores.txt", index=False)

print("Final MCDA output saved with", len(output), "rows.")


Matched rows: 6023
Calculating distance to nearest grid point...
Final MCDA output saved with 6023 rows.


In [5]:
# --- Final Map to locate optimal location with MCDA Analysis ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load MCDA results
df = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\FinalMCDAScore\final_mcda_scores.txt")

# Step 1: Filter for optimal zones (score ≥ 7.5)
optimal = df[df['final_score'] >= 7.5].copy()
print(f"Found {len(optimal)} optimal points with score ≥ 7")

# Step 2: Create map
m = folium.Map(location=[60, 10], zoom_start=5, tiles="CartoDB positron")

# Step 3: Use MarkerCluster to group close points
cluster = MarkerCluster().add_to(m)

for _, row in optimal.iterrows():
    popup = f"Score: {row['final_score']}<br>Solar: {row['solar_score']}<br>Wind: {row['wind_score']}<br>Land: {row['land_score']}<br>Grid: {row['grid_score']}"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup,
        icon=folium.Icon(color="green", icon="bolt", prefix='fa')
    ).add_to(cluster)

# Step 4: Save output
m.save(r"C:\GISDataManipulation\MCDA Files\FinalMCDAScore\Final_optimal_mcda_zones.html")
print("Optimal zones map saved to 'Final_optimal_mcda_zones.html'")


Found 21 optimal points with score ≥ 7
Optimal zones map saved to 'Final_optimal_mcda_zones.html'


In [2]:
# --- Final Map to locate top MCDA locations ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load MCDA results
df = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\FinalMCDAScore\final_mcda_scores.txt")

# Step 1: Get top 10 highest scoring locations
top_n = 10
top_locations = df.sort_values(by='final_score', ascending=False).head(top_n)
print(f"Top {top_n} locations selected from MCDA results.")

# Step 2: Create map
m = folium.Map(location=[60, 10], zoom_start=5, tiles="CartoDB positron")

# Step 3: Add clustered background of all high scoring points (optional)
high_score_points = df[df['final_score'] >= 7.5]
cluster = MarkerCluster().add_to(m)

for _, row in high_score_points.iterrows():
    popup = f"""<b>Final Score:</b> {row['final_score']}<br>
                Solar: {row['solar_score']}<br>
                Wind: {row['wind_score']}<br>
                Land: {row['land_score']}<br>
                Grid: {row['grid_score']}"""
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        color="blue",
        fill=True,
        fill_opacity=0.6,
        popup=popup
    ).add_to(cluster)

# Step 4: Add top-ranked locations with standout icons
for _, row in top_locations.iterrows():
    popup = f"""<b><u>Top Site</u></b><br>
                Score: {row['final_score']}<br>
                Solar: {row['solar_score']}<br>
                Wind: {row['wind_score']}<br>
                Land: {row['land_score']}<br>
                Grid: {row['grid_score']}"""
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup,
        icon=folium.Icon(color="red", icon="star", prefix="fa")
    ).add_to(m)

# Step 5: Save map
m.save(r"C:\GISDataManipulation\MCDA Files\FinalMCDAScore\Top10_MCDA_Locations_Map.html")
print("Top 10 MCDA locations map saved as: Top10_MCDA_Locations_Map.html")


Top 10 locations selected from MCDA results.
Top 10 MCDA locations map saved as: Top10_MCDA_Locations_Map.html


In [3]:
# --- Final Script to analyze optimal location for Energy Island with MCDA ---
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# --- Step 1: Load all datasets ---
solar = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\solar_mcda_scores.csv")
wind = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\wind_mcda_output.txt")
land = pd.read_csv(r"C:\GISDataManipulation\MCDA Files\landcover_mcda_scores.txt")
grid = pd.read_csv(r"C:\GISDataManipulation\StationTranmissionLineCombined\merged_grid_infra.txt")

# --- Step 2: Round coordinates and convert to numpy arrays ---
solar_coords = solar[['Latitude', 'Longitude']].round(5).to_numpy()
wind_coords = wind[['Latitude', 'Longitude']].round(5).to_numpy()
land_coords = land[['Latitude', 'Longitude']].round(5).to_numpy()

# --- Step 3: Match wind and land cover to solar using KDTree ---
tree_wind = cKDTree(wind_coords)
tree_land = cKDTree(land_coords)

_, idx_wind = tree_wind.query(solar_coords, k=1, distance_upper_bound=0.01)
_, idx_land = tree_land.query(solar_coords, k=1, distance_upper_bound=0.01)

# Filter valid matches
valid_mask = (idx_wind < len(wind)) & (idx_land < len(land))
solar = solar[valid_mask].reset_index(drop=True)
wind_matched = wind.iloc[idx_wind[valid_mask]].reset_index(drop=True)
land_matched = land.iloc[idx_land[valid_mask]].reset_index(drop=True)

# --- Step 4: Combine data, including PVOUT and Wind Speed ---
merged = pd.DataFrame({
    'Latitude': solar['Latitude'],
    'Longitude': solar['Longitude'],
    'solar_score': solar['solar_score'],
    'wind_score': wind_matched['wind_score'],
    'land_score': land_matched['land_score'],
    'PVOUT_Average': solar.get('PVOUT_Average', np.nan),
    'Wind_Speed': wind_matched.get('Wind_Speed', np.nan)
})

print("Matched rows:", len(merged))

# --- Step 5: Clean invalid coordinates ---
merged = merged.dropna(subset=['Latitude', 'Longitude'])
merged = merged[pd.to_numeric(merged['Latitude'], errors='coerce').notnull()]
merged = merged[pd.to_numeric(merged['Longitude'], errors='coerce').notnull()]
merged['Latitude'] = merged['Latitude'].astype(float)
merged['Longitude'] = merged['Longitude'].astype(float)

# --- Step 6: Compute distance to grid ---
print("Calculating distance to nearest grid point...")
def latlon_to_xy(lat, lon):
    return np.radians(lat) * 6371, np.radians(lon) * 6371 * np.cos(np.radians(lat))

sample_xy = np.array([latlon_to_xy(lat, lon) for lat, lon in zip(merged['Latitude'], merged['Longitude'])])
grid_xy = np.array([latlon_to_xy(lat, lon) for lat, lon in zip(grid['Latitude'], grid['Longitude'])])

tree = cKDTree(grid_xy)
dists, _ = tree.query(sample_xy, k=1)
merged['distance_to_grid_km'] = dists

# --- Step 7: Normalize and score grid distance ---
d_min = merged['distance_to_grid_km'].min()
d_max = merged['distance_to_grid_km'].max()
merged['grid_norm'] = (d_max - merged['distance_to_grid_km']) / (d_max - d_min)
merged['grid_score'] = (merged['grid_norm'] * 9).round().astype(int)

# --- Step 8: Compute final weighted MCDA score ---
merged['final_score'] = (
    0.35 * merged['solar_score'] +
    0.25 * merged['wind_score'] +
    0.20 * merged['land_score'] +
    0.20 * merged['grid_score']
).round(2)

# --- Step 9: Save results with actual PVOUT and Wind Speed ---
output = merged[['Latitude', 'Longitude', 'solar_score', 'wind_score', 'land_score',
                 'grid_score', 'final_score', 'PVOUT_Average', 'Wind_Speed']]

output_path = r"C:\GISDataManipulation\MCDA Files\FinalMCDAScore\final_mcda_scores.txt"
output.to_csv(output_path, index=False)

print("Final MCDA output saved with", len(output), "rows.")
print("Saved to:", output_path)


Matched rows: 6022
Calculating distance to nearest grid point...
Final MCDA output saved with 6022 rows.
Saved to: C:\GISDataManipulation\MCDA Files\FinalMCDAScore\final_mcda_scores.txt
